# CALCULATION OF FREQUENCY GRID FOR TESS LC PERIODOGRAM

This notebook calculates the desirable frequency grid for periodogram calculations, based on the characteristics of the TESS LC time series.

**Calculation details:**

For the establishment of the frequency grid for periodogram calculation, we need to define three values: the minimum frequency, the maximum frequency, and the frequency grid spacing, as per the recommendations on section 7.1 of _Understanding the Lomb–Scargle Periodogram (VanderPlas, J. T. 2018, APJS, 236(1), 16)_. In this work, the recommendations are the following ones:

- For the lower frequency limit, set it to 0, it would not do any difference in terms of computation load.
- For the upper frequency limit, set it to the Nyquist (uniform sampling) or pseudo-Nyquist (non-uniform sampling) or, even better, according to the specific scientific case (i.e. set it according to the periods we are searching for in the signals). 
- For the frequency grid spacing, use the inverse of the observation window, applying an additional factor between 1/5 and 1/10.

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $4 h$, so we could set the limits of the periodogram frequencies for periods between $5 min$ and $10 h$, so as to perfectly cover the target range and to leave enough margins if the stacked periodogram is to show some "plateau". In days, this corresponds to a range of $P\in[0.003472, 0.416667] d$ or, equivalently, frequencies in the range $f\in[288.0, 2.4] d^{-1}$.

For the grid spacing, we need to cover all the observation windows present in the objects under study, as they will all be different. We will hence calculate it looking into each of the source TESS LC files. For easier calculation, we consider just the overall observation time of objects, instead of the length of multiple different observation periods that exist in each object. In any case, the value used will be a maximum possible observation time, yielding the smallest possible grip spacing (i.e. more resolution). As opposed to the case of RV time series, there are a lot more points, and probably of more continuous observation time, so we will use the 1/10 factor.


## Modules and configuration

### Modules

In [1]:
# Modules import:
import pandas as pd

import lightkurve as lk

### Configuration

In [2]:
# Configuration:
# Files and folders (WARNING: THIS FOLDER STRUCTURE MUST EXIST PREVIOUSLY):
GTO_FILE = "../data/CARM_VIS_objects_with_PG.csv" # NOTE: initially this should be a copy of the previous file.

N0 = 10 # For the factor (1/N0 = 1/5, 1/10,...)


### Functions

In [3]:
def tess_lc_load(filename: str):
    '''Load the TESS LC file and returns a lightcurve object with just the three relevant columns
    Note: it seems to be necessary to drop 'nan' values for GLS to work properly.'''
    lc_lk = lk.read(filename).remove_nans()
    return lc_lk

## Data processing

### GTO data loading

In [4]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002958,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001986,NaN,NaN


In [5]:
gto.shape

(390, 248)

In [6]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Calculate the maximum observation window length present in the data

In [7]:
max_T = 0
correct_lc = 0
for i in range(0, len(gto)):
    try:
        # Read the TESS LC file:
        lc = tess_lc_load(gto.loc[i, 'lc_file'])
        # Calculate the observation time:
        T = lc.time.max().value - lc.time.min().value
        # Update the maximum:
        max_T = max(max_T, T)
        correct_lc += 1
    except:
        # Error, most probably because no LC file exists for this object.
        pass

print("Correctly processed files: %d" %correct_lc)
print("Max. RV observation time in days: %f" %max_T)

Correctly processed files: 271
Max. RV observation time in days: 27.417075


### Calculate the minimum frequency grid spacing

In [8]:
delta_f = 1.0 / (N0 * max_T)
print("Frequency grid spacing must be smaller than %f d^(-1)" %delta_f)

Frequency grid spacing must be smaller than 0.003647 d^(-1)


Hence, about a 0.001 frequency grid spacing should do well:

In [9]:
n = int((288.0 - 2.4) / 0.001)
print("Estimated number of points for each periodogram: %d" %n)

Estimated number of points for each periodogram: 285600


This is $\approx300000$ points for each periodogram, which seems reasonable, althougn in this case of TESS LC, the number of time points is much larger than in the cas of RV time series, so calculation will also take time, probably.

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- For the TESS LC periodogram calculations, we will set:
  - A lower frequency of 2.4 d^(-1).
  - An upper frequency of 288.0 d^(-1).
  - A grid spacing of 0.001 d^(-1). This would yield a total of 285600 frequency points to calculate for each periodogram.